In [38]:
import pandas as pd
import os
import tensorflow as tf
from sklearn.model_selection import train_test_split
import tensorflow_hub as hub



### Load and Explore birdclef-2024 data (pre conversion)

In [39]:
# Read train meta data
train_metadata_path = "../data/birdclef-2024/train_metadata.csv"

train_df = pd.read_csv(train_metadata_path)
train_df.head()

,primary_label,secondary_labels,type,latitude,longitude,scientific_name,common_name,author,license,rating,url,filename
0,asbfly,[],['call'],39.2297,118.1987,Muscicapa dauurica,Asian Brown Flycatcher,Matt Slaymaker,Creative Commons Attribution-NonCommercial-Sha...,5.0,https://www.xeno-canto.org/134896,asbfly/XC134896.ogg
1,asbfly,[],['song'],51.4030,104.6401,Muscicapa dauurica,Asian Brown Flycatcher,Magnus Hellström,Creative Commons Attribution-NonCommercial-Sha...,2.5,https://www.xeno-canto.org/164848,asbfly/XC164848.ogg
2,asbfly,[],['song'],36.3319,127.3555,Muscicapa dauurica,Asian Brown Flycatcher,Stuart Fisher,Creative Commons Attribution-NonCommercial-Sha...,2.5,https://www.xeno-canto.org/175797,asbfly/XC175797.ogg
3,asbfly,[],['call'],21.1697,70.6005,Muscicapa dauurica,Asian Brown Flycatcher,vir joshi,Creative Commons Attribution-NonCommercial-Sha...,4.0,https://www.xeno-canto.org/207738,asbfly/XC207738.ogg
4,asbfly,[],['call'],15.5442,73.7733,Muscicapa dauurica,Asian Brown Flycatcher,Albert Lastukhin & Sergei Karpeev,Creative Commons Attribution-NonCommercial-Sha...,4.0,https://www.xeno-canto.org/209218,asbfly/XC209218.ogg


In [40]:
train_df.describe()

,latitude,longitude,rating
count,24081.000000,24081.000000,24459.000000
mean,32.537040,43.640699,3.843493
std,19.440382,50.191352,1.100840
min,-43.524000,-171.765400,0.000000
25%,17.160100,2.545700,3.000000
50%,37.155100,26.687600,4.000000
75%,49.114400,85.319300,5.000000
max,71.964000,177.447800,5.000000


### Prepare dataframe pointing to bird call audio data in wav format

In [41]:
data_input_root = "../data/birdclef-2024/train_audio"
data_output_root = "../data/birdclef-2024/train_audio_16"

from nightingale.data_pipeline.audio_preprocessor import AudioPreprocessor
# convert data if necessary
preprocessor = AudioPreprocessor()

## Batch convert a complete dataset and store
# Read train meta data. Convert to yamnet compatible format. Skip if file already exists. Reproduce folder structure.
preprocessor.process_folder(
    input_root= data_input_root,
    output_root= data_output_root,
)

## Load single file
# input_audio_path = "../data/birdclef-2024/train_audio/cohcuc1/XC19645.wav"
# output_audio_path = "../XC19645.wav"
# audio_tensor = preprocessor.process_file(input_audio_path, output_audio_path)


Processing 4 folders
Skipped (already exists): ../data/birdclef-2024/train_audio_16/cohcuc1/XC214005.wav
Skipped (already exists): ../data/birdclef-2024/train_audio_16/cohcuc1/XC214006.wav
Skipped (already exists): ../data/birdclef-2024/train_audio_16/cohcuc1/XC19645.wav
Skipped (already exists): ../data/birdclef-2024/train_audio_16/cohcuc1/XC174932.wav
Skipped (already exists): ../data/birdclef-2024/train_audio_16/cohcuc1/XC177780.wav
Skipped (already exists): ../data/birdclef-2024/train_audio_16/cohcuc1/XC140254.wav
Skipped (already exists): ../data/birdclef-2024/train_audio_16/cohcuc1/XC191169.wav
Skipped (already exists): ../data/birdclef-2024/train_audio_16/cohcuc1/XC136837.wav
Skipped (already exists): ../data/birdclef-2024/train_audio_16/integr/XC842970.wav
Skipped (already exists): ../data/birdclef-2024/train_audio_16/integr/XC401712.wav
Skipped (already exists): ../data/birdclef-2024/train_audio_16/integr/XC397702.wav
Skipped (already exists): ../data/birdclef-2024/train_audio

### Filter the Metadata for the folder that are actually existing

In [42]:
from nightingale.data_pipeline.filter_birdclef_data import load_birdclef_metadata

# filter the metadata csv file for th data that is actually in the data folder
filtered_bird_df, num_of_bird_classes_in_dataset = load_birdclef_metadata(metadata_path=train_metadata_path, 
                                          audio_root="../data/birdclef-2024/train_audio_16")

print(f"Number of bird classes in dataset: {num_of_bird_classes_in_dataset}")

Number of bird classes in dataset: 3


### Split data: Training, Validation and Test

In [43]:
# Step 1: Split the data into training (60%), validation (20%) and test (20%) sets
train_df_idx, temp_df_idx = train_test_split(filtered_bird_df.index, test_size=0.4, random_state=42, stratify=filtered_bird_df['target'])
val_df_idx, test_df_idx = train_test_split(temp_df_idx, test_size=0.5, random_state=42, stratify=filtered_bird_df.loc[temp_df_idx, 'target'])

# Step 2: Create 'fold' column in original filtered_bird_df
filtered_bird_df['fold'] = ''  # initialize empty
filtered_bird_df.loc[train_df_idx, 'fold'] = 1
filtered_bird_df.loc[val_df_idx, 'fold'] = 2
filtered_bird_df.loc[test_df_idx, 'fold'] = 3

# filtered_bird_df.head(10)
# plt.hist(filtered_bird_df[filtered_bird_df['fold'] == 1]['target'], bins=len(bird_classes), alpha=0.7, label='Train')
# plt.hist(filtered_bird_df[filtered_bird_df['fold'] == 2]['target'], bins=len(bird_classes), alpha=0.7, label='Val')
# plt.hist(filtered_bird_df[filtered_bird_df['fold'] == 3]['target'], bins=len(bird_classes), alpha=0.7, label='Test')
# plt.xlabel('Bird Classes')
# plt.ylabel('Count')
# plt.title('Distribution of Bird Classes in Train, Val, and Test Sets')
# plt.legend()
# plt.show()

In [44]:
filenames_train = filtered_bird_df[filtered_bird_df['fold'] == 1]['filename']
targets_train = filtered_bird_df[filtered_bird_df['fold'] == 1]['target']

filenames_val = filtered_bird_df[filtered_bird_df['fold'] == 2]['filename']
targets_val = filtered_bird_df[filtered_bird_df['fold'] == 2]['target']

filenames_test = filtered_bird_df[filtered_bird_df['fold'] == 3]['filename']
targets_test = filtered_bird_df[filtered_bird_df['fold'] == 3]['target']


train_ds = tf.data.Dataset.from_tensor_slices((filenames_train, targets_train))
val_ds = tf.data.Dataset.from_tensor_slices((filenames_val, targets_val))
test_ds = tf.data.Dataset.from_tensor_slices((filenames_test, targets_test))

def load_wav_for_map(filename, label):
        """
        Read and decode a WAV file into a mono waveform tensor.

        Uses tf.io.read_file + tf.audio.decode_wav to load the file, squeezes the
        channel dimension so the returned tensor has shape [num_samples].

        Args:
          filename: path to the audio file (string tensor).
          label: associated label.

        Returns:
          (wav, label): tuple where wav is a 1-D float32 tensor and label is unchanged.
        """
        audio = tf.io.read_file(filename)
        wav, sr = tf.audio.decode_wav(audio, desired_channels=1)
        # Remove the trailing channel dimension -> shape [num_samples]
        wav = tf.squeeze(wav, axis=-1)
        return wav, label

train_ds = train_ds.map(load_wav_for_map)
val_ds = val_ds.map(load_wav_for_map)
test_ds = test_ds.map(load_wav_for_map)

# Load the Yamnet Model and extract Embeddings

In [45]:
model = hub.load('https://tfhub.dev/google/yamnet/1')
def extract_embedding(wav_data, label):
  ''' run YAMNet to extract embedding from the wav data '''
  scores, embeddings, spectrogram = model(wav_data)
  num_embeddings = tf.shape(embeddings)[0]
  return (embeddings,
            tf.repeat(label, num_embeddings))

In [46]:
train_ds = train_ds.map(extract_embedding).unbatch()
val_ds = val_ds.map(extract_embedding).unbatch()
test_ds = test_ds.map(extract_embedding).unbatch()
train_ds.element_spec

train_ds = train_ds.cache().shuffle(1000).batch(32).prefetch(tf.data.AUTOTUNE)
val_ds = val_ds.cache().batch(32).prefetch(tf.data.AUTOTUNE)
test_ds = test_ds.cache().batch(32).prefetch(tf.data.AUTOTUNE)

## Create Model for Classifier

In [47]:
from nightingale.model.classifier_head import ClassifierHead

bird_class_model = ClassifierHead(num_classes=num_of_bird_classes_in_dataset)

bird_class_model.summary()

Model: "classifier_head"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [48]:
bird_class_model.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                 optimizer="adam",
                 metrics=['accuracy'])

callback = tf.keras.callbacks.EarlyStopping(monitor='loss',
                                            patience=3,
                                            restore_best_weights=True)

# Train the model

#### Train classifier

In [49]:
history = bird_class_model.fit(train_ds,
                       epochs=5,
                       validation_data=val_ds,
                       callbacks=callback)

Epoch 1/5


/workspaces/nightingale/.venv/lib/python3.11/site-packages/keras/src/backend/tensorflow/nn.py:717: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Softmax activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


     14/Unknown 2s 4ms/step - accuracy: 0.6383 - loss: 0.7548

2025-11-25 12:36:22.157308: I tensorflow/core/framework/local_rendezvous.cc:407] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]
/workspaces/nightingale/.venv/lib/python3.11/site-packages/keras/src/trainers/epoch_iterator.py:164: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()


25/25 ━━━━━━━━━━━━━━━━━━━━ 4s 61ms/step - accuracy: 0.8582 - loss: 0.3638 - val_accuracy: 0.9267 - val_loss: 0.2084
Epoch 2/5
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9699 - loss: 0.1017 - val_accuracy: 0.9400 - val_loss: 0.1792
Epoch 3/5
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9812 - loss: 0.0634 - val_accuracy: 0.9444 - val_loss: 0.1663
Epoch 4/5
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9862 - loss: 0.0449 - val_accuracy: 0.9222 - val_loss: 0.1945


2025-11-25 12:36:23.534349: I tensorflow/core/framework/local_rendezvous.cc:407] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]
2025-11-25 12:36:23.617490: I tensorflow/core/framework/local_rendezvous.cc:407] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]
2025-11-25 12:36:23.734711: I tensorflow/core/framework/local_rendezvous.cc:407] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]


Epoch 5/5
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9912 - loss: 0.0343 - val_accuracy: 0.9378 - val_loss: 0.1777


#### Evaluate classifier

In [50]:
loss, accuracy = bird_class_model.evaluate(test_ds)

print("Loss: ", loss)
print("Accuracy: ", accuracy)

17/17 ━━━━━━━━━━━━━━━━━━━━ 1s 63ms/step - accuracy: 0.8969 - loss: 0.3787
Loss:  0.3786546289920807
Accuracy:  0.8968871831893921


# Create the MLFlow Server

In [51]:
import mlflow

mlflow.set_tracking_uri("http://0.0.0.0:5000")



In [36]:
import os
print("Env:", os.getenv("MLFLOW_TRACKING_URI"))
print("From MLflow:", mlflow.get_tracking_uri())

Env: http://0.0.0.0:5000
From MLflow: http://0.0.0.0:5000


### Create experiment 
RUN THE FOLLOWING CODE BLOCK ONLY ONCE FOR INITIAL EXPERIMENT SETUP!!

In [18]:
experiment_description = (
    "Nightingale is a bird call classification project."
)

experiment_tags = {
    "project_name": "nightingale",
    "mlflow.note.content": experiment_description,
}

# only run following command once to create the experiment after the server has been started for the first time
# client.create_experiment(name="Nightingale Bird Call Classification", tags=experiment_tags)
mlflow.set_experiment(
    experiment_name="nightingale"
)

<Experiment: artifact_location='/root/mlflow/artifacts/960222781255945012', creation_time=1763994490529, experiment_id='960222781255945012', last_update_time=1763994490529, lifecycle_stage='active', name='nightingale', tags={}>

In [19]:
# Assemble the metrics we're going to write into a collection
metrics = {"Loss": loss, "Accuracy": accuracy}
params = {
    "num_bird_classes": num_of_bird_classes_in_dataset,
    "optimizer": "adam",
    "loss_function": "SparseCategoricalCrossentropy",
    "loss_from_logits": True,
    "epochs": len(history.epoch),
    "batch_size": 32,
    "early_stopping_monitor": "loss",
    "early_stopping_patience": 3,
}

# Initiate the MLflow run context
with mlflow.start_run() as run:
    # Log the parameters used for the model fit
    mlflow.log_params(params)

    # Log the error metrics that were calculated during validation
    mlflow.log_metrics(metrics)

    # Take one batch from the dataset
    x_batch, y_batch = next(iter(train_ds))

    # Convert to numpy (MLflow expects numpy or tensor-like input, not a tf.data.Dataset)
    sample_input = x_batch.numpy()
    sample_output = bird_class_model.predict(sample_input)

    # Infer signature from data
    signature = mlflow.models.infer_signature(sample_input, sample_output)

    print("Shape of input_example:", sample_input.shape)
    # Log an instance of the trained model for later use
    model_info = mlflow.keras.log_model(model=bird_class_model, name = "Bird-Call-Classifier-Head", signature=signature, pip_requirements=['keras==3.10.0'], registered_model_name="nightingale-dev.default.Reg-Bird-Call-Classifier-Head")
#     # mlflow.sklearn.log_model(sk_model=rf, input_example=X_val, name=artifact_path)
    

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
Shape of input_example: (32, 1024)


2025/11/25 08:38:33 WARNING mlflow.utils.requirements_utils: Detected one or more mismatches between the model's dependencies and the current Python environment:
 - keras (current: 3.12.0, required: keras==3.10.0)
To fix the mismatches, call `mlflow.pyfunc.get_model_dependencies(model_uri)` to fetch the model's environment and install dependencies using the resulting environment file.
2025/11/25 08:38:33 WARNING mlflow.utils.environment: Failed to resolve installed pip version. ``pip`` will be added to conda.yaml environment spec without a version specifier.
Registered model 'nightingale-dev.default.Reg-Bird-Call-Classifier-Head' already exists. Creating a new version of this model...
2025/11/25 08:38:33 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: nightingale-dev.default.Reg-Bird-Call-Classifier-Head, version 2
Created version '2' of model 'nightingale-dev.default.Reg-Bird-Call-Classifier-Head'.


🏃 View run rambunctious-owl-571 at: http://0.0.0.0:5000/#/experiments/960222781255945012/runs/66a682e224794c09a32d93acf6d6e51a
🧪 View experiment at: http://0.0.0.0:5000/#/experiments/960222781255945012


#### Run inference on a bird call audio sample (YAMNet + classifier head)

In [ ]:
# wav = load_wav_16k_mono(filtered_bird_df[filtered_bird_df['fold'] == 3]['filename'].values[1])
# scores, embeddings, spectrogram = model(wav)
# result = bird_class_model(embeddings).numpy()

# inferred_class = bird_classes[result.mean(axis=0).argmax()]
# print(f'The main sound is: {inferred_class}')

In [ ]:
# bird_class_model.save('bird_classifier_head.keras')